FIRST APPROACH


In [3]:
from llama_index.core import PromptTemplate

choices = [
    "Related to Fee transparency: When we have question related to calcualtion of printing the dataframe or sum the fees between a time interval ",
    "Realted to Fees gpt: Vanilla question and annswer ,related to Fee schedule or some general questions",
]
def get_choice_str(choices):
    choices_str = "\n\n".join(
        [f"{idx+1}. {c}" for idx, c in enumerate(choices)]
    )
    return choices_str


choices_str = get_choice_str(choices)

In [4]:
router_prompt0 = PromptTemplate(
    "Some choices are given below. It is provided in a numbered list (1 to"
    " {num_choices}), where each item in the list corresponds to a"
    " summary.\n---------------------\n{context_list}\n---------------------\nUsing"
    " only the choices above and not prior knowledge, return the top choices"
    " (no more than {max_outputs}, but only select what is needed) that are"
    " most relevant to the question: '{query_str}'\n"
)

In [5]:
def get_formatted_prompt(query_str):
    fmt_prompt = router_prompt0.format(
        num_choices=len(choices),
        max_outputs=2,
        context_list=choices_str,
        query_str=query_str,
    )
    return fmt_prompt

In [7]:
import openai
import os
query_str = "sum fees in 2023"
fmt_prompt = get_formatted_prompt(query_str)
openai.api_key=os.getenv('OPENAI_KEY')
completion = openai.chat.completions.create(
  # Use GPT 3.5 as the LLM
  model="gpt-4",
  # Pre-define conversation messages for the possible roles 
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": fmt_prompt}
  ]
)
print(completion.choices[0].message.content)

1. Related to Fee transparency: When we have question related to calculation of printing the dataframe or sum the fees between a time interval


SECOND APPROACH

In [16]:
from dataclasses import fields
from pydantic import BaseModel,Field
import json

{'choice': {'title': 'Choice', 'description': 'An integer strictly 1 or 2,1 denotes related to Fee transparency and 2 denotes related to Fees Gpt', 'type': 'integer'}, 'reason': {'title': 'Reason', 'description': 'An explanation for the selected choice ', 'type': 'string'}}


In [17]:
choices = [
    "Related to Fee transparency: When we have question related to calcualtion of printing the dataframe or sum the fees between a time interval ",
    "Realted to Fees gpt: Vanilla question and annswer ,related to Fee schedule or some general questions",
]

In [18]:
from pydantic import Field
from typing import List
class Answer(BaseModel):
    choice: int= Field( description="An integer strictly 1 or 2,1 denotes related to Fee transparency and 2 denotes related to Fees Gpt")
    reason: str =Field( description="An explanation for the selected choice ")


class Answers(BaseModel):
    """Represents a list of answers."""

    answers: List[Answer]


In [19]:
from llama_index.program.openai import OpenAIPydanticProgram

In [22]:
router_prompt1 = PromptTemplate(
    "Some choices are given below. It is provided in a numbered list (1 to"
    " {num_choices}), where each item in the list corresponds to a"
    " summary.\n---------------------\n{context_list}\n---------------------\nUsing"
    " only the choices above and not prior knowledge, return the top choices"
    " (no more than {max_outputs}, but only select what is needed) that are"
    " most relevant to the question: '{query_str}'\n"
)

In [24]:
query_str = "Tell me about employee program at JP Morgan"
fmt= router_prompt1.partial_format(
    num_choices=len(choices),
    max_outputs=len(choices),
)
import openai
import os
openai.api_key=os.getenv('OPENAI_KEY')
program = OpenAIPydanticProgram.from_defaults(
    output_cls=Answers,
    prompt=fmt,
    verbose=True,
)
output = program(context_list=choices_str, query_str=query_str)

Function call: Answers with args: {"answers":[{"choice":2,"reason":"The choice 'Realted to Fees gpt' is selected because it mentions general questions, which could potentially include information about employee programs at JP Morgan."}]}
